# YOLO v3 Object Detection

Let's see how to use the state of the art in object detection! Please make sure to watch the video, there is no code along here, since we can't reasonably train the YOLOv3 network ourself, instead we will use a pre-established version.

CODE SOURCE: https://github.com/xiaochus/YOLOv3

REFERENCE (for original YOLOv3): 

        @article{YOLOv3,  
              title={YOLOv3: An Incremental Improvement},  
              author={J Redmon, A Farhadi },
              year={2018} 
--------
----------
## YOU MUST WATCH THE VIDEO LECTURE TO PROPERLY SET UP THE MODEL AND WEIGHTS. THIS NOTEBOOK WON'T WORK UNLESS YOU FOLLOW THE EXACT SET UP SHOWN IN THE VIDEO LECTURE.
-------
-------

In [ ]:
pip install tensorflow==1.15

    100% |################################| 124.0MB 100kB/s ta 0:00:0113% |#                               | 4.6MB 3.9MB/s eta 0:00:31    15% |####                            | 19.2MB 3.4MB/s eta 0:00:31    20% |######                          | 25.4MB 3.8MB/s eta 0:00:27    29% |#########                       | 36.9MB 1.6MB/s eta 0:00:54    50% |################                | 63.2MB 3.8MB/s eta 0:00:16    52% |################                | 64.8MB 3.2MB/s eta 0:00:19    80% |#########################       | 100.2MB 3.9MB/s eta 0:00:07    96% |##############################  | 120.1MB 3.2MB/s eta 0:00:02    97% |############################### | 120.3MB 5.4MB/s eta 0:00:01
    100% |################################| 51kB 6.9MB/s ta 0:00:011
    100% |################################| 51kB 7.0MB/s ta 0:00:011
    100% |################################| 3.8MB 1.7MB/s ta 0:00:011  4% |#                               | 153kB 5.3MB/s eta 0:00:01    45% |##############               

In [2]:
pip install --upgrade numpy

    100% |################################| 15.3MB 597kB/s ta 0:00:01   14% |####                            | 2.2MB 3.8MB/s eta 0:00:04    29% |#########                       | 4.6MB 3.4MB/s eta 0:00:04
  Found existing installation: numpy 1.15.1
    Uninstalling numpy-1.15.1:
      Successfully uninstalled numpy-1.15.1
You are using pip version 10.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install tensorflow-gpu keras

You are using pip version 10.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
python-3.6 -m pip install fish

SyntaxError: invalid syntax (<ipython-input-1-08bb2710a1a6>, line 1)

In [1]:
import os
import time
import cv2
import numpy as np
from model.yolo_model import YOLO

Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [1]:
def process_image(img):
    """Resize, reduce and expand image.

    # Argument:
        img: original image.

    # Returns
        image: ndarray(64, 64, 3), processed image.
    """
    image = cv2.resize(img, (416, 416),
                       interpolation=cv2.INTER_CUBIC)
    image = np.array(image, dtype='float32')
    image /= 255.
    image = np.expand_dims(image, axis=0)

    return image

In [3]:
def get_classes(file):
    """Get classes name.

    # Argument:
        file: classes name for database.

    # Returns
        class_names: List, classes name.

    """
    with open(file) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]

    return class_names

In [4]:
def draw(image, boxes, scores, classes, all_classes):
    """Draw the boxes on the image.

    # Argument:
        image: original image.
        boxes: ndarray, boxes of objects.
        classes: ndarray, classes of objects.
        scores: ndarray, scores of objects.
        all_classes: all classes name.
    """
    for box, score, cl in zip(boxes, scores, classes):
        x, y, w, h = box

        top = max(0, np.floor(x + 0.5).astype(int))
        left = max(0, np.floor(y + 0.5).astype(int))
        right = min(image.shape[1], np.floor(x + w + 0.5).astype(int))
        bottom = min(image.shape[0], np.floor(y + h + 0.5).astype(int))

        cv2.rectangle(image, (top, left), (right, bottom), (255, 0, 0), 2)
        cv2.putText(image, '{0} {1:.2f}'.format(all_classes[cl], score),
                    (top, left - 6),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (0, 0, 255), 1,
                    cv2.LINE_AA)

        print('class: {0}, score: {1:.2f}'.format(all_classes[cl], score))
        print('box coordinate x,y,w,h: {0}'.format(box))

    print()

In [5]:
def detect_image(image, yolo, all_classes):
    """Use yolo v3 to detect images.

    # Argument:
        image: original image.
        yolo: YOLO, yolo model.
        all_classes: all classes name.

    # Returns:
        image: processed image.
    """
    pimage = process_image(image)

    start = time.time()
    boxes, classes, scores = yolo.predict(pimage, image.shape)
    end = time.time()

    print('time: {0:.2f}s'.format(end - start))

    if boxes is not None:
        draw(image, boxes, scores, classes, all_classes)

    return image

In [6]:
def detect_video(video, yolo, all_classes):
    """Use yolo v3 to detect video.

    # Argument:
        video: video file.
        yolo: YOLO, yolo model.
        all_classes: all classes name.
    """
    video_path = os.path.join("videos", "test", video)
    camera = cv2.VideoCapture(video_path)
    cv2.namedWindow("detection", cv2.WINDOW_AUTOSIZE)

    # Prepare for saving the detected video
    sz = (int(camera.get(cv2.CAP_PROP_FRAME_WIDTH)),
        int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    fourcc = cv2.VideoWriter_fourcc(*'mpeg')

    
    vout = cv2.VideoWriter()
    vout.open(os.path.join("videos", "res", video), fourcc, 20, sz, True)

    while True:
        res, frame = camera.read()

        if not res:
            break

        image = detect_image(frame, yolo, all_classes)
        cv2.imshow("detection", image)

        # Save the video frame by frame
        vout.write(image)

        if cv2.waitKey(110) & 0xff == 27:
                break

    vout.release()
    camera.release()
    

In [7]:
yolo = YOLO(0.6, 0.5)
file = 'data/coco_classes.txt'
all_classes = get_classes(file)

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '25607' (I am process '36989')



You can find the C code in this temporary file: /var/folders/cv/xs_npjzj5ng6dhqvfmg1zpg40000gn/T/theano_compilation_error_b2ywg25b


Exception: ('The following error happened while compiling the node', DotModulo(A, s, m, A2, s2, m2), '\n', 'Compilation failed (return status=1): ld: unknown option: -platform_version. clang-10: error: linker command failed with exit code 1 (use -v to see invocation). ', '[DotModulo(A, s, m, A2, s2, m2)]')

### Detecting Images

In [10]:
f = 'jingxiang-gao-489454-unsplash.jpg'
path = 'images/'+f
image = cv2.imread(path)
image = detect_image(image, yolo, all_classes)
cv2.imwrite('images/res/' + f, image)

time: 3.36s
class: person, score: 0.60
box coordinate x,y,w,h: [2523.36770296 1482.90817547  621.14238739 1302.1294961 ]
class: bicycle, score: 0.84
box coordinate x,y,w,h: [2877.7012825  2007.04590225 1301.56508088  721.28467417]



True

# Detecting on Video

In [9]:
# # detect videos one at a time in videos/test folder    
# video = 'library1.mp4'
# detect_video(video, yolo, all_classes)

time: 3.23s
class: person, score: 0.60
box coordinate x,y,w,h: [2523.36770296 1482.90817547  621.14238739 1302.1294961 ]
class: bicycle, score: 0.84
box coordinate x,y,w,h: [2877.7012825  2007.04590225 1301.56508088  721.28467417]



True